In [3]:
from microscope import *
import ipywidgets as widgets

%matplotlib inline

In [15]:
l_1 = 1064e-9
l_2 = 532e-9
N_POINTS = 128  # Resolution of image
pixel_size = 1e-10

input_coordinate_system = CoordinateSystem(dxdydz=(pixel_size, pixel_size), n_points=(N_POINTS, N_POINTS))
first_wave = WaveFunction(psi=np.ones((N_POINTS, N_POINTS)),
                          coordinates=input_coordinate_system,
                          E0=Joules_of_keV(300))

dummy_sample = SamplePropagator(dummy_potential='letters small',
                                axes=tuple([first_wave.coordinates.axes[0],
                                            first_wave.coordinates.axes[1],
                                            np.linspace(-5e-10, 5e-10, 2)]))

first_lens = LensPropagator(focal_length=3.3e-3, fft_shift=True)
second_lens = LensPropagator(focal_length=3.3e-3, fft_shift=False)

E_1_default = find_amplitude_for_phase(starting_E=1e7, cavity_type='analytical', plot=False, print_progress=False, NA_1=0.05)

In [18]:
def f(NA_1, defocus, Cs, n_electrons, ring_cavity, E_1):
    cavity = CavityAnalyticalPropagator(l_1=l_1, l_2=l_2, E_1=E_1, NA_1=NA_1)
    aberration_propagator = AberrationsPropagator(Cs=Cs, defocus=defocus, atigmatism_parameter=0, astigmatism_orientation=0)
    
    M_2f_a = Microscope([dummy_sample, first_lens, cavity, second_lens, aberration_propagator], n_electrons=n_electrons)
    pic_2f_a, _ = M_2f_a.take_a_picture(first_wave)
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    
    ax[0].imshow(pic_2f_a)
    ax[0].colorbar()
    ax[0].xlabel("x")
    ax[0].ylabel("z")
    

widgets.interact(f, NA_1=(0.001, 0.2, 0.01), defocus=(0, 1e-21, 3e-23), Cs=(0, 1e-7, 1e-9), n_electrons=(0, 1e8, 1e7), ring_cavity=True, E_1=(E_1_default / 5, E_1_default * 5, E_1_default / 4));

interactive(children=(FloatSlider(value=0.091, description='NA_1', max=0.2, min=0.001, step=0.01), FloatSlider…